# Trong trường hợp chạy trên kaggle, commentate phần đọc từ đường dẫn VS Code và ngược lại

In [6]:
import pandas as pd
import requests
import time

# # Config cho kaggle
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [7]:
# # Đọc từ đường dẫn kaggle
# file_path = '/kaggle/input/full_movie_data.csv'
# df = pd.read_csv(file_path)

# df_update = df[df['Status'] == 'Ongoing']['Title']
# df_update

In [8]:
# Đọc từ đường dẫn VS Code
df = pd.read_csv('../data/raw/full_movie_data.csv')
df_update = df[df['Status'] == 'Ongoing']['Title']
df_update

C:\Users\kdat1\AppData\Local\Temp\ipykernel_5928\158983895.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/full_movie_data.csv')


3579                             Paris
4560               The Ascent of Money
6077     Louis Theroux: Law & Disorder
7023                         Dreileben
7634            Dragon Age: Redemption
                     ...              
92411                           Troppo
92413                      Zombieverse
92415                     Wolf Like Me
92418                    Above Justice
92428        The Angry Video Game Nerd
Name: Title, Length: 880, dtype: object

In [9]:
df_tv = pd.DataFrame()

for page in range(1,5):
    # URL API
    url = "https://api.themoviedb.org/3/tv/on_the_air?"
    params = {
        "api_key": "234da40a2c2c86ddae7089772282eae4",
        "language": "en-US",
        "page": page
        
    }

    # Gửi yêu cầu GET tới API
    response = requests.get(url, params=params)

    # Kiểm tra nếu yêu cầu thành công
    if response.status_code == 200:
        data = response.json()
        # Trích xuất danh sách các phim từ kết quả JSON
        movies = data.get('results', [])
        
        # Chuyển dữ liệu thành DataFrame
        df_tv = pd.concat([df_tv, pd.DataFrame(movies)])

    # URL API
    url = "https://api.themoviedb.org/3/tv/airing_today?"
    params = {
        "api_key": "234da40a2c2c86ddae7089772282eae4",
        "language": "en-US",
        "page": page
    }

    # Gửi yêu cầu GET tới API
    response = requests.get(url, params=params)

    # Kiểm tra nếu yêu cầu thành công
    if response.status_code == 200:
        data = response.json()
        # Trích xuất danh sách các phim từ kết quả JSON
        movies = data.get('results', [])
        
        # Chuyển dữ liệu thành DataFrame
        df_tv = pd.concat([df_tv, pd.DataFrame(movies)])

df_update = pd.concat([df_update, df_tv['name']])
df_update

3579                            Paris
4560              The Ascent of Money
6077    Louis Theroux: Law & Disorder
7023                        Dreileben
7634           Dragon Age: Redemption
                    ...              
15        Shinkalion Change the World
16                 Match of the Day 2
17                          Marry YOU
18                      Carte Blanche
19       Sunday with Laura Kuenssberg
Length: 1040, dtype: object

In [10]:
df_mv = pd.DataFrame()

for page in range(1,5):
    # URL API
    url = "https://api.themoviedb.org/3/movie/now_playing?"
    params = {
        "api_key": "234da40a2c2c86ddae7089772282eae4",
        "language": "en-US",
        "page": page
    }

    # Gửi yêu cầu GET tới API
    response = requests.get(url, params=params)

    # Kiểm tra nếu yêu cầu thành công
    if response.status_code == 200:
        data = response.json()
        # Trích xuất danh sách các phim từ kết quả JSON
        movies = data.get('results', [])
        
        # Chuyển dữ liệu thành DataFrame
        df_mv = pd.concat([df_mv, pd.DataFrame(movies)])

    # URL API
    url = "https://api.themoviedb.org/3/movie/upcoming?"
    params = {
        "api_key": "234da40a2c2c86ddae7089772282eae4",
        "language": "en-US",
        "page": page
    }

    # Gửi yêu cầu GET tới API
    response = requests.get(url, params=params)

    # Kiểm tra nếu yêu cầu thành công
    if response.status_code == 200:
        data = response.json()
        # Trích xuất danh sách các phim từ kết quả JSON
        movies = data.get('results', [])
        
        # Chuyển dữ liệu thành DataFrame
        df_mv = pd.concat([df_mv, pd.DataFrame(movies)])

df_update = pd.concat([df_update, df_mv['title']])
df_update

3579                            Paris
4560              The Ascent of Money
6077    Louis Theroux: Law & Disorder
7023                        Dreileben
7634           Dragon Age: Redemption
                    ...              
15                             Pin/Ya
16                            Cabrini
17                          Lembayung
18                             Sumala
19                         Parthenope
Length: 1200, dtype: object

In [11]:
api_key_list = ['8f5aa155', 'dbd262c8', '8a9f3511', 'b0ead728','2d918779', '817e9847', '6d85304', 'c05a60d1', '7c616a06', 'f8711aaf', '8e22b5b']

In [12]:
def get_movie_data(title, api_key):
    url = f"http://www.omdbapi.com/?t={title}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  # Trả về dữ liệu JSON từ OMDB API
    else:
        return None

In [13]:
# Chia DataFrame thành các khối với mỗi khối 1000 dòng
def crawl_movies_by_api_keys(df, api_key_list):
    start = 0
    chunk_size = 1000
    for i, api_key in enumerate(api_key_list):
        # Xác định phần của DataFrame dựa trên vị trí
        chunk = df.iloc[start:start + chunk_size]
        
        # Nếu chunk rỗng, nghĩa là hết dữ liệu để crawl
        if chunk.empty:
            break
        
        # Crawl thông tin phim cho từng imdbId trong chunk
        for title in chunk:
            movie_info = get_movie_data(title, api_key)
            if movie_info and movie_info.get('Response') == 'True':
                movie_data.append(movie_info)
        
        # Cập nhật vị trí bắt đầu cho chunk tiếp theo
        start += chunk_size

In [14]:
# Danh sách để lưu thông tin phim
movie_data = []

# Gọi hàm để crawl dữ liệu phim
crawl_movies_by_api_keys(df_update, api_key_list)

# Chuyển đổi dữ liệu sang DataFrame
df_movies = pd.DataFrame(movie_data)

status = []
for i in df_movies['Year']:
    if '–1' in i or '–2' in i or '–' not in i: status.append('Finished')
    else: status.append('Ongoing') 
df_movies['Status'] = status

df = pd.concat([df, df_movies])
df = df.drop_duplicates(subset= ['Title', 'imdbID'], keep= 'last')

# Lưu dữ liệu phim vào file CSV mới
output_file = '../data/raw/full_movie_data.csv'
df.to_csv(output_file, index=False)

print(f"Đã lưu thông tin phim vào file: {output_file}")

Đã lưu thông tin phim vào file: ../data/raw/full_movie_data.csv
